In [1]:
import os

TRAIN_PATH = "ibug_300W_large_face_landmark_dataset/labels_ibug_300W_train_eyes.xml"
TEST_PATH = "ibug_300W_large_face_landmark_dataset/labels_ibug_300W_test_eyes.xml"

In [2]:
# define the path to the temporary model file
TEMP_MODEL_PATH = "temp.dat"
# define the path to the output CSV file containing the results of
# our experiments
CSV_PATH = "trials.csv"
# define the path to the example image we'll be using to evaluate
# inference speed using the shape predictor
IMAGE_PATH = "example.jpg"
# define the number of threads/cores we'll be using when trianing our
# shape predictor models
PROCS = -1
# define the maximum number of trials we'll be performing when tuning
# our shape predictor hyperparameters
MAX_TRIALS = 100

In [3]:
# import the necessary packages
# from pyimagesearch import config
from sklearn.model_selection import ParameterGrid
import multiprocessing
import numpy as np
import random
import time
import dlib
import cv2

In [4]:
def evaluate_model_acc(xmlPath, predPath):
    # compute and return the error (lower is better) of the shape
    # predictor over our testing path
    return dlib.test_shape_predictor(xmlPath, predPath)

In [5]:
def evaluate_model_speed(predictor, imagePath, tests=10):
    # initialize the list of timings
    timings = []
    # loop over the number of speed tests to perform
    for i in range(0, tests):
        # load the input image and convert it to grayscale
        image = cv2.imread(config.IMAGE_PATH)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # detect faces in the grayscale frame
        detector = dlib.get_frontal_face_detector()
        rects = detector(gray, 1)
        # ensure at least one face was detected
        if len(rects) > 0:
            # time how long it takes to perform shape prediction
            # using the current shape prediction model
            start = time.time()
            shape = predictor(gray, rects[0])
            end = time.time()
        # update our timings list
        timings.append(end - start)
    # compute and return the average over the timings
    return np.average(timings)

In [6]:
# define the columns of our output CSV file
cols = [
	"tree_depth",
	"nu",
	"cascade_depth",
	"feature_pool_size",
	"num_test_splits",
	"oversampling_amount",
	"oversampling_translation_jitter",
	"inference_speed",
	"training_time",
	"training_error",
	"testing_error",
	"model_size"
]

In [7]:
# open the CSV file for writing and then write the columns as the
# header of the CSV file

csv = open(CSV_PATH, "w")
csv.write("{}\n".format(",".join(cols)))
# determine the number of processes/threads to use
procs = multiprocessing.cpu_count()
procs = PROCS if PROCS > 0 else procs

In [8]:
# initialize the list of dlib shape predictor hyperparameters that
# we'll be tuning over
hyperparams = {
    "tree_depth": list(range(2, 8, 2)),
    "nu": [0.01, 0.1, 0.25],
    "cascade_depth": list(range(6, 16, 2)),
    "feature_pool_size": [100, 250, 500, 750, 1000],
    "num_test_splits": [20, 100, 300],
    "oversampling_amount": [1, 20, 40],
    "oversampling_translation_jitter": [0.0, 0.1, 0.25]
}

In [9]:
# construct the set of hyperparameter combinations and randomly
# sample them as trying to test *all* of them would be
# computationally prohibitive
combos = list(ParameterGrid(hyperparams))
random.shuffle(combos)
sampledCombos = combos[:MAX_TRIALS]
print("[INFO] sampling {} of {} possible combinations".format(
    len(sampledCombos), len(combos)))

[INFO] sampling 100 of 6075 possible combinations


In [ ]:
# loop over our hyperparameter combinations
for (i, p) in enumerate(sampledCombos):
    # log experiment number
    print("[INFO] starting trial {}/{}...".format(i + 1,len(sampledCombos)))

    # grab the default options for dlib's shape predictor and then
    # set the values based on our current hyperparameter values
    options = dlib.shape_predictor_training_options()
    options.tree_depth = p["tree_depth"]
    options.nu = p["nu"]
    options.cascade_depth = p["cascade_depth"]
    options.feature_pool_size = p["feature_pool_size"]
    options.num_test_splits = p["num_test_splits"]
    options.oversampling_amount = p["oversampling_amount"]
    otj = p["oversampling_translation_jitter"]
    options.oversampling_translation_jitter = otj
    # tell dlib to be verbose when training and utilize our supplied
    # number of threads when training
    options.be_verbose = True
    options.num_threads = procs
    
    
    start = time.time()
    dlib.train_shape_predictor(TRAIN_PATH,TEMP_MODEL_PATH, options)
    trainingTime = time.time() - start
    break